# How to add standard tests to a tool

When creating either a custom tool or a new tool to publish in a LangChain integration, it is important to add standard tests to ensure the tool works as expected. This guide will show you how to add standard tests to a tool.

## Setup

First, let's install 2 dependencies:

- `langchain-core` will define the interfaces we want to import to define our custom tool.
- `langchain-standard-tests` (installed as a `git` dependency) will provide the standard tests we want to use.

:::note
The `langchain-standard-tests` package is not released to pypi, so we need to install it from the git repository.

This will always pull in the latest version of the package.
:::

In [ ]:
%pip install -U langchain-core "git+https://github.com/langchain-ai/langchain.git#subdirectory=libs/standard-tests"


Let's start with the custom tool defined in the first part of [how to create tools](./custom_tools.ipynb).

In [1]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b